In [11]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad
from PIL import Image
modes = [AES.MODE_ECB, AES.MODE_CBC, AES.MODE_OFB, AES.MODE_CFB, AES.MODE_CTR]
mode_names = {AES.MODE_ECB: 'ECB', AES.MODE_CBC: 'CBC', AES.MODE_OFB: 'OFB', AES.MODE_CFB: 'CFB', AES.MODE_CTR: 'CTR'}
files = ['maly_pingwinek.png', 'sredni_pingwinek.png', 'duzy_pingwinek.png']
key = get_random_bytes(16)

In [12]:
def byte_xor(bytes1: bytes, bytes2: bytes):
    return bytes(b1 ^ b2 for b1, b2 in zip(bytes1, bytes2))

In [18]:
for mode in modes:
    cipher = AES.new(key=key, mode=mode)
    for file in files:
        img = Image.open(file)
        byte_img = img.tobytes()
        byte_img = pad(byte_img, AES.block_size)
        en_bytes = cipher.encrypt(byte_img)
        en_png = Image.frombytes(img.mode, img.size, en_bytes)
        en_png.save('pingwiny/normal/{}_{}'.format(mode_names[mode], file), 'PNG')

In [19]:
for mode in modes:
    cipher = AES.new(key=key, mode=mode)
    err_cipher = AES.new(key=key, mode=mode)
    for file in files:
        img = Image.open(file)
        byte_img = img.tobytes()
        byte_img = pad(byte_img, AES.block_size)
        en_bytes = cipher.encrypt(byte_img)
        error = bytes.fromhex('0001')
        err_bytes = byte_xor(en_bytes[:2], error) + en_bytes[2:]
        de_bytes = err_cipher.decrypt(err_bytes)
        de_png = Image.frombytes(img.mode, img.size, de_bytes)
        de_png.save('pingwiny/file_error/file_error_{}_{}'.format(mode_names[mode], file), 'PNG')

In [20]:
err_key = byte_xor(key, (bytes.fromhex('0001') + b'\0' * 14))
print(key, err_key)
for mode in modes:
    cipher = AES.new(key=key, mode=mode)
    err_cipher = AES.new(key=err_key, mode=mode)
    for file in files:
        img = Image.open(file)
        byte_img = img.tobytes()
        byte_img = pad(byte_img, AES.block_size)
        en_bytes = cipher.encrypt(byte_img)
        de_bytes = err_cipher.decrypt(en_bytes)
        de_png = Image.frombytes(img.mode, img.size, de_bytes)
        de_png.save('pingwiny/key_error/key_error_{}_{}'.format(mode_names[mode], file), 'PNG')

b'\xf8\xcbq\x8b\xec\xba\xf3\xf5lQ\x94\x10\x00\xcd>\x8b' b'\xf8\xcaq\x8b\xec\xba\xf3\xf5lQ\x94\x10\x00\xcd>\x8b'


In [16]:
def cbc_encrypt(data, key, iv):
    cipher = AES.new(key=key, mode=AES.MODE_ECB)
    c_bytes = b''
    prev_block = iv
    
    for i in range(0, len(data), AES.block_size):
        new_block = data[i : i + AES.block_size]
        xor_block = byte_xor(new_block, prev_block)
        en_block = cipher.encrypt(xor_block)
        prev_block = en_block
        c_bytes += en_block
    
    return c_bytes        

def cbc_decrypt(data, key, iv):
    cipher = AES.new(key=key, mode=AES.MODE_ECB)
    c_bytes = b''
    prev_block = iv
    
    for i in range(0, len(data), AES.block_size):
        new_block = data[i : i + AES.block_size]
        de_block = cipher.decrypt(new_block)
        xor_block = byte_xor(de_block, prev_block)
        c_bytes += xor_block
        prev_block = new_block
    
    return c_bytes       

In [17]:
iv = get_random_bytes(AES.block_size)
byte_img = get_random_bytes(10_000)
print(byte_img)
byte_img = pad(byte_img, AES.block_size)
en_bytes = cbc_encrypt(byte_img, key, iv)
print(en_bytes)
de_bytes = cbc_decrypt(en_bytes, key, iv)
print(de_bytes)
print(len(de_bytes))

b'\xbac<\x0ep\xe0KB\xa0\xa2\x85=\xec|\xcdaS\xb4\xcc\xd1\x03\xab\x1d\x84\x1b\x91x\xb6.\xc4\xec\xf0Y\xbc\xc6\xae|H\xe5\xbf\xac1\x943\x0c[\xf1\xbcL\xe1\x15)\xd3\xbbBRu\xda}\xd2GY\x80$,P\xaf\x92k\xcaa5?\\(\xc6K\x12\xd0\x02~\x1c oe\xa4\x7fqW\xb2\xebx\xbd\x80!\xc7\xd1\xddn|Y\xecusF\x18q\t\xe3\xde~\x9e\x03\xb0\xd6K\x92\xd0P\x82\xbb_\xf4\x16\xb8o\xe3\xc5\xec\xac\x88F\xd9\xca\xee!\xd4\x9fV\xb1\xa9\x93Jv1\xb5F\x85\x85\xfb#\x924\xfc*\x1d\xf3\xaecK\xd7* \x1d\x05\xc4\x9a\x08\xc3\x88=\xd6\'\xc3\xaf\x97\xc8\x89\x8b\x9e\xba\xf2\xc7\xf5\xef\xb0E\xbfm"\xef\x12\xbb\x19*TX\x0c\r]\xf8\xa2\x02n\xb9P\xa1\x1c\xd4\xa8<BH\x0c\xe0@\xd5\xa3]\xdd\xa2\xab\x8fJ\x0c\xb0\x19\x1c\x04\xb2\xbf\xe6y@\x81|\x0b\xb9\x15BCl\xc7\xe3n\xe8I\x82\x96\xbe\xf0+,+@\xcc+\x80\xee\xe5>\x13\x1d#\xbe@\xd6cS\xd3\x93S\x92q\x0c\xb6\xbc\xab\x1f\xa3N\x84\xfe \xdcXj\xdd\xd5\xeca\x99e\xb1\xf6\x1c\xf8,u.O|l&\x92?\xfb\xc5\x00R\x08P[U\x949\xf9\xb7N\xfdY\\\x1c\x8eS\xa6#7\xf3\x8f\x1b\xde\x86a\xd7\x075\xe0\x94TlP\\\xb3 \x9e\xb2\xcfX\x8f$)\x08\x9f/\xae